# Configuration

> Working with config file.

In [ ]:
#default_exp config

In [ ]:
#export
from pathlib import Path
import configparser

def get_config(cfgpath=None):
    if cfgpath is not None:
        cfgpath = Path(cfgpath)
        assert cfgpath.exists(), f'Config not found at "{cfgpath}"'
        cfg = configparser.ConfigParser()
        cfg.read(cfgpath)
        print(f'Read config from "{cfgpath}"')
        return cfg
    
    cfgname = 'settings.ini'
    try:
        d = Path(__file).parent
        proj = d.name
    except NameError:
        d = Path().absolute()
        proj = None
    while not (d/cfgname).exists() and (d != d.parent):
        d = d.parent
    return get_config(d/cfgname)

In [ ]:
c = get_config()
assert c.get('DEFAULT', 'lib_name') == 'reseng'